In [18]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [33]:
url = 'https://lhr.nu.edu.pk/faculty/'
r = requests.get(url)
print(r)

<Response [200]>


In [34]:
soup = BeautifulSoup(r.content, 'html.parser')

# Iterate for each faculty card

In [35]:
id_=[]
name=[]
designation=[]
phdSupervisor=[]
email=[]
dept=[]
ext=[]
img=[]
highesteducation=[]

In [36]:
f = soup.findAll('div', class_='facultyCard')
len(f)

184

In [37]:
for i in range(len(f)):
    faculty=f[i]
    
    ######################
    facultyId = faculty.find('a', class_='faculty-link')['href'].strip('/')[-4:]
    facultyName = faculty.find('h5', class_='text-center').text
    facultyDesignation = faculty.find('p', class_='small').text.strip().split('\n')[0]
    isSupervisor = (faculty.find('p', class_='small').text.strip().split('\n')[-1] == 
    '                          HEC Approved PhD Supervisor')
    facultyEmail = faculty.find('p', class_='mb-0').text
    facultyDepartment = faculty.find_previous('h1', class_='mb-2 mt-3').text.split('Faculty')[0]
    facultyIMG = 'https://lhr.nu.edu.pk' + faculty.find('img', class_='card-img-top')['src']
    
    # FIND EDUCATION AND EXTENSION
    facultyLink = 'https://lhr.nu.edu.pk' + faculty.find('a', class_='faculty-link')['href']
        
    re = requests.get(facultyLink)
    soup2 = BeautifulSoup(re.content, 'html.parser')
    
    facultyEducation = None  # Initialize to None
    education_div = soup2.find('div', class_='col-lg-8')
    if education_div:
        ul = education_div.find_next('ul')
        if ul:
            li = ul.find_next('li')
            if li and li.text:
                facultyEducation = li.text.split(',')[0]
                
    facultyExtension = None
    extension_span = soup2.find('span', class_='small')
    if extension_span:
        facultyExtension = extension_span.text.split('Ext:')[-1]
        
    #######################
    
    ## APPEND TO EACH LIST
    try:
        id_.append(int(facultyId))
    except:
        id_.append(-1)
        
    name.append(str(facultyName))
    designation.append(str(facultyDesignation))
    phdSupervisor.append(bool(isSupervisor))
    email.append(facultyEmail)
    dept.append(str(facultyDepartment))
    
    try:
        ext.append(int(facultyExtension))
    except:
        ext.append(-1)
        
    img.append(facultyIMG)
    highesteducation.append(str(facultyEducation))

In [38]:
df = pd.DataFrame({'ID':id_,
                   'Name':name,
                   'Designation':designation,
                   'HEC Approved PHD Supervisor':phdSupervisor,
                   'Highest Education':highesteducation,
                   'Email':email,
                   'Department':dept,
                   'Extension':ext,
                   'ImageURL':img})

In [39]:
df

,ID,Name,Designation,HEC Approved PHD Supervisor,Highest Education,Email,Department,Extension,ImageURL
0,1238,Dr. Kashif Zafar,Professor & HOD,True,Ph.D(Computer Science),kashif.zafar@nu.edu.pk,FAST School of Computing,221,https://lhr.nu.edu.pk/media/Faculty/02_-_Dr._K...
1,4391,Dr. Asif Mahmood Gillani,Professor,True,Ph.D(Computing),asif.gilani@nu.edu.pk,FAST School of Computing,291,https://lhr.nu.edu.pk/media/Faculty/03_-_Dr._A...
2,5424,Dr. Hammad Naveed,Professor,True,Ph.D. (Bioengineering),hammad.naveed@nu.edu.pk,FAST School of Computing,227,https://lhr.nu.edu.pk/media/Faculty/01_-_Dr._H...
3,4027,Dr. Aamir Wali,Associate Professor,True,PhD (CS),aamir.wali@nu.edu.pk,FAST School of Computing,260,https://lhr.nu.edu.pk/media/Faculty/44_-_Dr._A...
4,6174,Dr. Arshad Ali,Associate Professor,True,PhD (CS),arshad.ali1@nu.edu.pk,FAST School of Computing,406,https://lhr.nu.edu.pk/media/Faculty/04_-_Dr._A...
...,...,...,...,...,...,...,...,...,...
179,6420,Ms. Nokhaiz Zahra,Instructor,False,MA,nokhaiz.zahra@nu.edu.pk,Department of Science & Humanities,324,https://lhr.nu.edu.pk/media/Faculty/165_-_Nokh...
180,6181,Ms. Rida Ahmed,Instructor,False,BS (English),rida.ahmed@nu.edu.pk,Department of Science & Humanities,363,https://lhr.nu.edu.pk/media/Faculty/6181-remov...
181,6555,Ms. Aqsa Naz,Lecturer (English),False,M.Phil (English),aqsa.naz@nu.edu.pk,Department of Science & Humanities,324,https://lhr.nu.edu.pk/media/Faculty/161_-_Aqsa...
182,-1,Mr. Hafiz Umair Gulzar,Lecturer (Islamiat),False,M.Phil (Islamic Studies),umair.gulzar@nu.edu.pk,Department of Science & Humanities,207,https://lhr.nu.edu.pk/media/Faculty/163_-_Hafi...


In [40]:
df.to_csv('lhr.csv')

In [ ]:
# df['HEC Approved PHD Supervisor'].value_counts()

In [ ]:
# df[df['Highest Education'].isnull()]

In [ ]:
# df.duplicated()

# Testing for one faculty card

In [10]:
faculty = soup.findAll('div', class_='facultyCard')[0]
faculty

<div class="card facultyCard card-br-0 ml-0 mr-0 pb-0">
<div class="facultyImage">
<img class="card-img-top rounded-circle mt-3 mb-0 d-block mx-auto" src="/media/Faculty/02_-_Dr._Kashif_Zafar_Prof__HoD_CS.png"/>
</div>
<div class="card-body pr-0 pl-0 pb-0">
<h5 class="text-center">Dr. Kashif Zafar</h5>
<p class="small text-center font-italic">Professor &amp; HOD
                        
                         <br> HEC Approved PhD Supervisor 
                        
                        </br></p>
<p class="mb-0 text-center">kashif.zafar@nu.edu.pk</p>
<!--  <p class="mt-0 small text-center">ext no: 221</p> -->
<a class="faculty-link" href="/fsc/facultyProfile/1238"></a>
</div>
</div>

In [11]:
facultyId = faculty.find('a', class_='faculty-link')['href'].strip('/')[-4:]

In [12]:
facultyName = faculty.find('h5', class_='text-center').text

In [13]:
facultyDesignation = faculty.find('p', class_='small').text.strip().split('\n')[0]

In [14]:
isSupervisor = (faculty.find('p', class_='small').text.strip().split('\n')[-1] == 
'                          HEC Approved PhD Supervisor')
#isSupervisor

In [15]:
# facultyEmail = faculty.find('p', class_='mb-0').text

In [17]:
department = faculty.find_previous('h1', class_='mb-2 mt-3').text.split('Faculty')[0]
department

'FAST School of Computing '

In [12]:
# facultyLink = 'https://lhr.nu.edu.pk' + faculty.find('a', class_='faculty-link')['href']

In [13]:
# facultyIMG = 'https://lhr.nu.edu.pk' + faculty.find('img', class_='card-img-top')['src']

In [14]:
# profile_url=facultyLink
# re = requests.get(profile_url)
# soup = BeautifulSoup(re.content, 'html.parser')
# soup.find('div', class_='col-lg-8').find_next('ul').find_next('li').text.split(',')[0]

In [15]:
# re = requests.get(facultyLink)
# soup3 = BeautifulSoup(re.content, 'html.parser')
# int(soup3.find('span', class_='small').text.split('Ext:')[-1])

## Profile page to extract highest education

In [4]:
# def highest_edu(profile_url):
#     re = requests.get(profile_url)
#     soup = BeautifulSoup(re.content, 'html.parser')
#     high_ed = soup.find('div', class_='col-lg-8')
    
#     # Check if the element was found
#     if high_ed:
#         ul = high_ed.find_next('ul')
        
#         # Check if the <ul> tag was found
#         if ul:
#             li = ul.find_next('li')
            
#             # Check if the <li> tag was found
#             if li and li.text:
#                 return li.text.split(',')[0]
    
#     # Return a default value (or None) if any part of the extraction fails
#     return None